In [59]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import json

import unicodedata
import re

In [60]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [ ]:
start_at = 1
end_at = 13

In [62]:
client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
  )

In [63]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [64]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [65]:
chinese_extraction_prompt = "The provided content is a Chinese translation of the Ainu original text in Markdown. Please extract the Chinese translation. Only extract the numbered items."
english_extraction_prompt = "The provided content is an English translation of the Ainu original text in Markdown. Please extract the English translation. Only extract the numbered items."

In [66]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [ ]:
# process the translation in batch mode

for song_no in range(start_at, end_at + 1):

    md_name_part = get_output_file_name_key(ainu_titles[song_no - 1])

    with open(f"Chiri_footnotes/footnote_story_{song_no}.txt", "r", encoding="utf8") as f:
        japanese_story = f.read()
        japanese_story = unicodedata.normalize('NFKC', japanese_story)

    with open(f"LLM_prompts_and_raw_translations_footnotes/Chinese_Translation/{song_no}_{md_name_part}_to_Chinese.md", "r", encoding="utf8") as f:
        md_chinese_text = f.read()
        md_chinese_text = unicodedata.normalize('NFKC', md_chinese_text)

        chinese_translation_extraction = generate(client,generate_content_config,model,input_text=md_chinese_text,prompt=chinese_extraction_prompt)

    with open(f"LLM_prompts_and_raw_translations_footnotes/English_Translation/{song_no}_{md_name_part}_to_English.md", "r", encoding="utf8") as f:
        md_english_text = f.read()
        md_english_text = unicodedata.normalize('NFKC', md_english_text)

        english_translation_extraction = generate(client,generate_content_config,model,input_text=md_english_text,prompt=english_extraction_prompt)



    data_dict = {'ainu_title': ainu_titles[song_no - 1], 'japanese_title': japanese_titles[song_no], 'japanese_translation': japanese_story, 'chinese_translation': chinese_translation_extraction, 'english_translation': english_translation_extraction}

    with open(f"raw_translations_footnotes_json/{song_no}_{md_name_part}.json", "w", encoding="utf8") as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)



(1) isoeonkami.iso 是海幸,eonkami 是......謝罪之意。
鯨魚擱淺在岸邊,是因為海神大人為了賜予人類,親自帶來並推上岸的,人們深信不疑,此時一定會由德高望重的人盛裝打扮,面向大海的方向進行禮拜。
(2) ononno. 這是出海或上山打獵的人,帶著獵物歸來時,迎接的人們異口同聲說出的話語。
(3) uniwente......大水災、火災、火山爆發等各種天災之後,或是有人被熊吃掉、落入海或河中,以及其他任何因變故而受傷或死亡的情況下所舉行的儀式。
屆時,人們會互相用槍或刀尖抵住對方,同時交換慰問之詞。一個村莊若有災民,鄰近村莊會派出許多代表聚集到該村莊舉行儀式,即使只有一個人也會舉行。
(4) hokokse......uniwente 時,或發生重大變故時,向神明祈求救助時男性的呼喊聲。フオホホーイ,僅限男性。
(5) ashur 是變故的消息,ek 是到來。
......從村莊遠方出外旅行的人,生病或死亡時,由該地派遣使者將變故告知其故鄉,或是從其他村莊有人來告知某某人過世了,或發生了某某變故等等,人們將這種情況稱為 ashur。
該使者被稱為 ashurkorkur(帶有變故消息的人)。
ashurkorkur 在靠近村莊時,首先會大聲呼喊 hokokse(フオホホーイ)。聽到呼喊聲的村民,也會大聲呼喊,並到村莊邊緣迎接,詢問變故的消息。
(6) uchishkar......互相哭泣。這是女性的寒暄方式,久別重逢時、近親過世時、有人遭遇重大危險好不容易脫險時等等,女性之間會手牽著手,或互相擁抱頭部或肩膀哭泣。
(7) matrimimse(女性的呼喊聲)......發生緊急情況或 uniwente 的情況時,男性會發出粗獷的 hokokse(フオホホーイ)聲,女性則會發出細微的ホーイ聲。
據說女性的聲音比男性的聲音更高亢響亮,因此能先傳入神明的耳中。因此,發生緊急變故時,即使是男性也會像女性一樣發出細微的聲音,呼喊兩三聲。
(8) peutanke......與 rimimse 相同的意思,但這是普通常用的詞彙,rimimse 則稍微是比較難的詞彙。
1.  isoeonkami.iso refers to the sea, eonkami refers to the act of apologizing for somethi